In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import time
import numpy as np
import plotly.graph_objs as go

# df = pd.read_csv(
#     '/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Code/Undergraduate-Thesis/Preprocessing_Code/final/data/proportions.csv', low_memory=False)
# officers = pd.read_csv(
#     '/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Code/Undergraduate-Thesis/Preprocessing_Code/final/data/perm_unique_officers.csv', low_memory=False)

df = pd.read_csv('/content/gdrive/MyDrive/Thesis_Data/proportions.csv')
officers = pd.read_csv('/content/gdrive/MyDrive/Thesis_Data/perm_unique_officers.csv')

In [ ]:
print(df.columns)
print(df.head())
print(officers.columns)

Index(['CRID', 'OfficerID', 'OfficeFirst', 'OfficerLast', 'Category',
       'Allegation', 'Beat', 'IncidentDate', 'Beat Count', 'Beat Proportion'],
      dtype='object')
     CRID  OfficerID OfficeFirst OfficerLast      Category  \
0  259002       3055   Cornelius       Brown  Use Of Force   
1  259002      19347     Kenneth     Molesky  Use Of Force   
2  259011      10864        Mark   Grohovena  Use Of Force   
3  259013      20651       James     Norwood  Use Of Force   
4  259013       3055   Cornelius       Brown  Use Of Force   

                               Allegation    Beat         IncidentDate  \
0      Excessive Force / On Duty - Injury  2221.0  2000-01-01 00:00:00   
1      Excessive Force / On Duty - Injury  2221.0  2000-01-01 00:00:00   
2  Excessive Force / Off Duty - No Injury   925.0  2000-01-01 00:00:00   
3     Excessive Force / Off Duty - Injury  2232.0  2000-01-01 00:00:00   
4      Excessive Force / On Duty - Injury  2232.0  2000-01-01 00:00:00   

   Beat Cou

In [ ]:
# choose only OfficerID and Beat columns
prep_df = df[['OfficerID', 'Beat', 'Beat Count', 'Beat Proportion']]
prep_officers = officers[['OfficerID', 'Beat']]
# group by CRID
df_grouped = df[['CRID', 'OfficerID', 'Beat', 'Beat Count', 'Beat Proportion']].groupby('CRID')
# make df_grouped into a numpy array
np_grouped = df_grouped.apply(lambda x: x.to_numpy())
# make np_grouped into a numpy array
np_grouped = np_grouped.to_numpy()

In [ ]:
# make df into numpy
np_df = prep_df.to_numpy()
np_officers = prep_officers.to_numpy()
# make array dtype object float
np_df = np_df.astype(float)
np_officers = np_officers.astype(float)

In [ ]:
# Column 0 is OfficerID, Column 1 is Beat
# ----------- Start of Original -----------
# get value counts of OfficerID
officer_counts = np.unique(np_df[:, 0], return_counts=True)
# get number of times each value appears in officer_counts
# sort in descending order
officer_counts = np.unique(officer_counts[1], return_counts=True)
# if number of complaints exceed 20, make all of the counts into a single bin
# make a copy of the original array to save for reference
officer_counts_copy = np.copy(officer_counts)
print(officer_counts)
# this is to prevent the graph from being too stretched
for i in range(len(officer_counts[0])):
    if officer_counts[0][i] > 10:
        officer_counts[1][9] += officer_counts[1][i]
        print(officer_counts[1][i])
        officer_counts[1][i] = 0
officer_counts = np.delete(officer_counts, np.s_[10:], axis=1)
# plot density curve
print(officer_counts)
fig = go.Figure()
fig.add_trace(go.Scatter(x=officer_counts[0], y=officer_counts[1], mode='lines', name='Data'))
fig.update_layout(title_text='Original Data')
fig.show()

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 35]), array([3593, 1734, 1006,  658,  404,  264,  193,  119,  111,   57,   47,
         41,   26,   13,   13,   12,    9,    4,    7,    5,    2,    1,
          1,    1,    1,    2,    1,    1,    1,    1]))
47
41
26
13
13
12
9
4
7
5
2
1
1
1
1
2
1
1
1
1
[[   1    2    3    4    5    6    7    8    9   10]
 [3593 1734 1006  658  404  264  193  119  111  246]]


In [ ]:
# print sum of counts
print('Sum of Counts: ', np.sum(officer_counts[1]))

Sum of Counts:  8328


In [ ]:
num_simulations = 1000

In [ ]:
# ----------- Start of Permutation Test -----------
# permutation test loop 1000 simulations
start_time = time.time()
# perm_test_results to be a 3d array
perm_test_results = []
for i in range(num_simulations):
    print(i)
    # make 1d array of 0s, length of np_officers
    perm_test = np.array([])
    # for each element in np_grouped, get length of each individual element
    for incident in np_grouped:
        # get length of each individual element
        num_officers = len(incident)
        # get beat of group
        beat = incident[0][2]
        if beat not in np_officers[:, 1]:
            continue
        # get officers in beat
        officers_in_beat = np_officers[np_officers[:, 1] == beat]
        if len(officers_in_beat) < num_officers:
            continue
        # randomly sample officers without replacement from officers in beat
        sample_officers = np.random.choice(officers_in_beat[:, 0], num_officers, replace=False)
        # add sample officers to perm_test
        perm_test = np.append(perm_test, sample_officers)
    # get value counts of OfficerID in perm_test
    perm_test_counts = np.unique(perm_test, return_counts=True)
    # print(perm_test_counts)
    # get number of times each value appears in perm_test_counts
    # sort in descending order
    perm_test_counts = np.unique(perm_test_counts[1], return_counts=True)
    # add perm_test_counts to perm_test_results
    perm_test_results.append(perm_test_counts)
end_time = time.time()
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
print(end_time - start_time)
# print(perm_test_results)

890.5983817577362


In [ ]:
# make perm_test_results into a pandas df
perm_test_results = pd.DataFrame(perm_test_results)
# rename columns
perm_test_results.columns = ['Number of Complaints', 'Counts']
# get proportion of each number of complaints by going through each value of number of complaints one by one

In [ ]:
perm_test_results.head(10)

,Number of Complaints,Counts
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1520, 1982, 1754, 1287, 680, 331, 127, 50, 22..."
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[1562, 1930, 1845, 1234, 695, 315, 135, 54, 13..."
2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1509, 1970, 1782, 1236, 693, 329, 151, 53, 12..."
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[1503, 2033, 1815, 1227, 665, 320, 136, 53, 20..."
4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1556, 1992, 1712, 1270, 681, 345, 133, 55, 16..."
5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1520, 1953, 1824, 1241, 665, 312, 153, 51, 20..."
6,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[1496, 1972, 1769, 1269, 689, 340, 128, 44, 21..."
7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1526, 2007, 1765, 1220, 697, 322, 135, 68, 13..."
8,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[1469, 1954, 1829, 1259, 716, 313, 124, 49, 9,..."
9,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[1551, 1992, 1779, 1242, 672, 326, 127, 70, 12..."


In [ ]:
# add 0 to each datapoint in Number of Complaints
perm_test_results['Number of Complaints'] = perm_test_results['Number of Complaints'].apply(lambda x: np.concatenate(([0], x)))
# and insert the np.sum(officer_counts[1]) - np.sum(polya_urn_results['Counts']) into the second column
perm_test_results['Counts'] = perm_test_results['Counts'].apply(lambda x: np.concatenate(([np.sum(officer_counts[1]) - np.sum(x)], x)))
# add 1 to all the counts to perm_test_results
perm_test_results['Number of Complaints'] = perm_test_results['Number of Complaints'].apply(lambda x: x + 1)

In [ ]:
perm_test_results.head(10)

,Number of Complaints,Counts
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[569, 1520, 1982, 1754, 1287, 680, 331, 127, 5..."
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]","[538, 1562, 1930, 1845, 1234, 695, 315, 135, 5..."
2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[587, 1509, 1970, 1782, 1236, 693, 329, 151, 5..."
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[546, 1503, 2033, 1815, 1227, 665, 320, 136, 5..."
4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[560, 1556, 1992, 1712, 1270, 681, 345, 133, 5..."
5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[578, 1520, 1953, 1824, 1241, 665, 312, 153, 5..."
6,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[592, 1496, 1972, 1769, 1269, 689, 340, 128, 4..."
7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[566, 1526, 2007, 1765, 1220, 697, 322, 135, 6..."
8,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[594, 1469, 1954, 1829, 1259, 716, 313, 124, 4..."
9,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[546, 1551, 1992, 1779, 1242, 672, 326, 127, 7..."


In [ ]:
# for each row in pem_test_results, print out sum of counts column
perm_sum = []
for i in range(len(perm_test_results)):
    perm_sum.append(sum(perm_test_results.iloc[i, 1]))

print(np.mean(perm_sum))

8328.0


In [ ]:
# explode the number of complaints column, then get max value
max_complaints = max(perm_test_results['Number of Complaints'].explode())
print(max_complaints)
# create dictionary of number of complaints and counts, key is number of complaints, value is counts
# keys should range from 1 to max_complaints
complaints_dict = {}
for i in range(1, 11):
    complaints_dict[i] = []
# for each row in perm_test_results, get the list of Number of Complaints, then get the list of Counts, then map them to the dictionary
for index, row in perm_test_results.iterrows():
    # get list of number of complaints
    complaints = row['Number of Complaints']
    # get list of counts
    counts = row['Counts']
    # map them to the dictionary
    for i in range(len(complaints)):
        if complaints[i] < 10:
            complaints_dict[complaints[i]].append(counts[i])
        else:
            complaints_dict[10].append(counts[i])

# if the list length is not equal to num_simulations, then add 0s to the list until it is equal to num_simulations
for key in complaints_dict:
    if len(complaints_dict[key]) != num_simulations:
        complaints_dict[key] = complaints_dict[key] + [0] * (num_simulations - len(complaints_dict[key]))

17


In [ ]:
# get the average of each list in the dictionary
perm_avg_complaints_dict = {}
for key in complaints_dict:
    perm_avg_complaints_dict[key] = sum(complaints_dict[key])/num_simulations
print(perm_avg_complaints_dict)

{1: 569.338, 2: 1502.729, 3: 1997.849, 4: 1795.917, 5: 1233.26, 6: 689.492, 7: 326.884, 8: 136.291, 9: 51.054, 10: 25.186}


In [ ]:
# get the 95% confidence interval of each list in the dictionary
upper_perm_ci_complaints_dict = {}
lower_perm_ci_complaints_dict = {}
for key in complaints_dict:
    # 95% confidence interval using quantile
    upper_perm_ci_complaints_dict[key] = np.quantile(complaints_dict[key], 0.975)
    lower_perm_ci_complaints_dict[key] = np.quantile(complaints_dict[key], 0.025)

In [ ]:
# plot curve
# Number of Complaints are on the x-axis, Counts are on the y-axis
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(perm_avg_complaints_dict.keys()), y=list(perm_avg_complaints_dict.values()), mode='lines', name='Permutation Test Results'))
# add line with color
fig.add_trace(go.Scatter(x=list(upper_perm_ci_complaints_dict.keys()), y=list(upper_perm_ci_complaints_dict.values()), mode='lines', name='Permutation Test 95% Confidence Interval', line=dict(color='rgb(235, 52, 58)', dash='dash')))
fig.add_trace(go.Scatter(x=list(lower_perm_ci_complaints_dict.keys()), y=list(lower_perm_ci_complaints_dict.values()), mode='lines', name='Permutation Test 95% Confidence Interval', line=dict(color='rgb(235, 52, 58)', dash='dash'), fill='tonexty', fillcolor='rgba(235, 52, 58,0.2)'))
fig.add_trace(go.Scatter(x=officer_counts[0], y=officer_counts[1], mode='lines', name='Data'))
fig.update_layout(title_text='Permutation Test Results')
fig.show()

In [ ]:
# ----------- Start of Polya Urn Model -----------
weighted_np_officers = np_officers
# add 0 column to weighted_np_officers
weighted_np_officers = np.insert(weighted_np_officers, 2, 0, axis=1)
weighted_np_officers = np.insert(weighted_np_officers, 3, 1, axis=1)
# sort weighted_np_officers by values in second column
weighted_np_officers = weighted_np_officers[weighted_np_officers[:, 1].argsort()]
# for each unique value in the second column, get the number of times it appears
unique_beats, beat_counts = np.unique(weighted_np_officers[:, 1], return_counts=True)
# for each unique beat, make the sum of the third column equal to 1
for beat in unique_beats:
    # get indices of beat
    beat_indices = np.where(weighted_np_officers[:, 1] == beat)
    # get number of officers in beat
    num_officers = beat_counts[np.where(unique_beats == beat)]
    # get weights for officers in beat
    weighted_np_officers[beat_indices, 2] = 1 / num_officers

polya_urn_results = []
start_time = time.time()
alpha = 0.20
for i in range(num_simulations):
    print(i)
    # make 1d array of 0s, length of np_officers
    polya_urn = np.array([])
    temp_weighted_np_officers = weighted_np_officers.copy()
    # for each element in np_grouped, get length of each individual element
    for incident in np_grouped:
        # get length of each individual element
        num_officers = len(incident)
        # get beat of group
        beat = incident[0][2]
        if beat not in weighted_np_officers[:, 1]:
            continue
        # get officers in beat
        officers_in_beat = temp_weighted_np_officers[temp_weighted_np_officers[:, 1] == beat]
        if len(officers_in_beat) < num_officers:
            continue
        # randomly sample officers without replacement from officers in beat
        p = officers_in_beat[:, 2] / sum(officers_in_beat[:, 2])
        sample_officers = np.random.choice(officers_in_beat[:, 0], num_officers, replace=False, p=p)
        # add sample officers to polya_urn
        polya_urn = np.append(polya_urn, sample_officers)
        # print(sample_officers)
        # selected officers weights go up by alpha
        for officer in sample_officers:
            temp_weighted_np_officers[temp_weighted_np_officers[:, 0] == officer, 2] += alpha
            temp_weighted_np_officers[temp_weighted_np_officers[:, 0] == officer, 3] += 1
        # if officer allegations go above 35, set weight to 0
        # temp_weighted_np_officers[temp_weighted_np_officers[:, 3] > 35, 2] = 0.000000001
    # get value counts of OfficerID in polya_urn
    polya_urn_counts = np.unique(polya_urn, return_counts=True)
    # get number of times each value appears in polya_urn_counts
    # sort in descending order
    polya_urn_counts = np.unique(polya_urn_counts[1], return_counts=True)
    # add polya_urn_counts to polya_urn_results
    polya_urn_results.append(polya_urn_counts)
end_time = time.time()
print(end_time - start_time)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
polya_urn_results = pd.DataFrame(polya_urn_results)
polya_urn_results.columns = ['Number of Complaints', 'Counts']

In [ ]:
# add 0 to each datapoint in Number of Complaints
polya_urn_results['Number of Complaints'] = polya_urn_results['Number of Complaints'].apply(lambda x: np.concatenate(([0], x)))
# and insert the np.sum(officer_counts[1]) - np.sum(polya_urn_results['Counts']) into the second column
polya_urn_results['Counts'] = polya_urn_results['Counts'].apply(lambda x: np.concatenate(([np.sum(officer_counts[1]) - np.sum(x)], x)))
# add 1 to all the counts to match
polya_urn_results['Number of Complaints'] = polya_urn_results['Number of Complaints'].apply(lambda x: x + 1)

In [ ]:
# Remark, Data here is ascending, number of complaints seems like it gets capped, will need to look into this
polya_urn_results.head(50)

,Number of Complaints,Counts
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4926, 790, 443, 358, 255, 193, 178, 150, 156,..."
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4992, 730, 441, 334, 281, 182, 187, 156, 116,..."
2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4946, 759, 440, 347, 270, 220, 176, 151, 117,..."
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4967, 779, 470, 324, 244, 213, 174, 137, 135,..."
4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4956, 726, 481, 300, 271, 221, 199, 170, 115,..."
5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4895, 764, 461, 364, 271, 227, 176, 161, 127,..."
6,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4931, 753, 476, 349, 282, 203, 179, 152, 121,..."
7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4921, 791, 425, 325, 276, 227, 186, 148, 133,..."
8,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4939, 764, 472, 368, 257, 220, 167, 128, 109,..."
9,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4873, 798, 467, 350, 281, 191, 179, 171, 121,..."


In [ ]:
# explode the number of complaints column, then get max value
urn_max_complaints = max(polya_urn_results['Number of Complaints'].explode())
print(urn_max_complaints)
# create dictionary of number of complaints and counts, key is number of complaints, value is counts
urn_complaints_dict = {}
for i in range(1, 11):
    urn_complaints_dict[i] = []
# for each row in polya_urn_results, get the list of Number of Complaints, then get the list of Counts, then map them to the dictionary
for index, row in polya_urn_results.iterrows():
    # get list of number of complaints
    complaints = row['Number of Complaints']
    # get list of counts
    counts = row['Counts']
    # map them to the dictionary
    for i in range(len(complaints)):
        if complaints[i] < 10:
            urn_complaints_dict[complaints[i]].append(counts[i])
        else:
            urn_complaints_dict[10].append(counts[i])

# if the list length is not equal to num_simulations, then add 0s to the list until it is equal to num_simulations
for key in urn_complaints_dict:
    if len(urn_complaints_dict[key]) != num_simulations:
        urn_complaints_dict[key] = urn_complaints_dict[key] + [0] * (num_simulations - len(urn_complaints_dict[key]))

114


In [ ]:
# get the average of each list in the dictionary
urn_avg_complaints_dict = {}
for key in urn_complaints_dict:
    urn_avg_complaints_dict[key] = np.mean(urn_complaints_dict[key])
print(urn_avg_complaints_dict)

{1: 4927.119, 2: 765.223, 3: 465.598, 4: 339.66, 5: 265.057, 6: 215.35, 7: 178.081, 8: 150.307, 9: 128.239, 10: 19.973305312108746}


In [ ]:
# get the 95% confidence interval of each list in the dictionary
upper_urn_ci_complaints_dict = {}
lower_urn_ci_complaints_dict = {}
for key in urn_complaints_dict:
    # 95% confidence interval using quantile
    upper_urn_ci_complaints_dict[key] = np.quantile(urn_complaints_dict[key], 0.975)
    lower_urn_ci_complaints_dict[key] = np.quantile(urn_complaints_dict[key], 0.025)

In [ ]:
# plot curve
# Number of Complaints are on the x-axis, Counts are on the y-axis
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(urn_avg_complaints_dict.keys()), y=list(urn_avg_complaints_dict.values()), mode='lines', name='Polya Urn Model Results'))
# add line with color
fig.add_trace(go.Scatter(x=list(upper_urn_ci_complaints_dict.keys()), y=list(upper_urn_ci_complaints_dict.values()), mode='lines', name='Polya Urn Model 95% Confidence Interval', line=dict(color='rgb(66, 81, 245)', dash='dash')))
fig.add_trace(go.Scatter(x=list(lower_urn_ci_complaints_dict.keys()), y=list(lower_urn_ci_complaints_dict.values()), mode='lines', name='Polya Urn Model 95% Confidence Interval', line=dict(color='rgb(66, 81, 245)', dash='dash'), fill='tonexty', fillcolor='rgba(66, 81, 245,0.2)'))
fig.add_trace(go.Scatter(x=officer_counts[0], y=officer_counts[1], mode='lines', name='Data'))
fig.update_layout(title_text='Polya Urn Model Results')
fig.show()

In [ ]:
# ----------- Start of Spillover Polya Urn Model -----------
weighted_np_officers = np_officers
# add 0 column to weighted_np_officers
weighted_np_officers = np.insert(weighted_np_officers, 2, 0, axis=1)
weighted_np_officers = np.insert(weighted_np_officers, 3, 1, axis=1)
# sort weighted_np_officers by values in second column
weighted_np_officers = weighted_np_officers[weighted_np_officers[:, 1].argsort()]
# for each unique value in the second column, get the number of times it appears
unique_beats, beat_counts = np.unique(weighted_np_officers[:, 1], return_counts=True)
# for each unique beat, make the sum of the third column equal to 1
for beat in unique_beats:
    # get indices of beat
    beat_indices = np.where(weighted_np_officers[:, 1] == beat)
    # get number of officers in beat
    num_officers = beat_counts[np.where(unique_beats == beat)]
    # get weights for officers in beat
    weighted_np_officers[beat_indices, 2] = 1 / num_officers

polya_urn_results = []
start_time = time.time()
# alpha is the proportion that an officer's weight is increased by per complaint they recieve
alpha = 0.20
# beta is the spillover proportion of which an officer's weight is increased based on other officers involved in the same incident
beta = 0.4
for i in range(num_simulations):
    print(i)
    # make 1d array of 0s, length of np_officers
    polya_urn = np.array([])
    temp_weighted_np_officers = weighted_np_officers.copy()
    # for each element in np_grouped, get length of each individual element
    for incident in np_grouped:
        # get length of each individual element
        num_officers = len(incident)
        # get beat of group
        beat = incident[0][2]
        if beat not in weighted_np_officers[:, 1]:
            continue
        # get officers in beat
        officers_in_beat = temp_weighted_np_officers[temp_weighted_np_officers[:, 1] == beat]
        if len(officers_in_beat) < num_officers:
            continue
        # randomly sample officers without replacement from officers in beat
        p = officers_in_beat[:, 2] / sum(officers_in_beat[:, 2])
        sample_officers = np.random.choice(officers_in_beat[:, 0], num_officers, replace=False, p=p)
        # add sample officers to polya_urn
        polya_urn = np.append(polya_urn, sample_officers)
        # print(sample_officers)
        # selected officers weights go up by alpha
        for officer in sample_officers:
            temp_weighted_np_officers[temp_weighted_np_officers[:, 0] == officer, 2] += alpha
            temp_weighted_np_officers[temp_weighted_np_officers[:, 0] == officer, 3] += 1
        # other officers in sample_officers weights go up by (1 - beta) / (len(sample_officers) - 1) * (sum of officer new weights)
        for officer in sample_officers:
            if sample_officers.size == 1:
                break
            other_officers = sample_officers[sample_officers != officer]
            for other_officer in other_officers:
                temp_weighted_np_officers[temp_weighted_np_officers[:, 0] == other_officer, 2] = beta * temp_weighted_np_officers[temp_weighted_np_officers[:, 0] == other_officer, 2] + (1 - beta) / (len(sample_officers) - 1) * temp_weighted_np_officers[temp_weighted_np_officers[:, 0] == officer, 2]
    # get value counts of OfficerID in polya_urn
    polya_urn_counts = np.unique(polya_urn, return_counts=True)
    # get number of times each value appears in polya_urn_counts
    # sort in descending order
    polya_urn_counts = np.unique(polya_urn_counts[1], return_counts=True)
    # add polya_urn_counts to polya_urn_results
    polya_urn_results.append(polya_urn_counts)
end_time = time.time()
print(end_time - start_time)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
polya_urn_results = pd.DataFrame(polya_urn_results)
polya_urn_results.columns = ['Number of Complaints', 'Counts']

In [ ]:
# add 0 to each datapoint in Number of Complaints
polya_urn_results['Number of Complaints'] = polya_urn_results['Number of Complaints'].apply(lambda x: np.concatenate(([0], x)))
# and insert the np.sum(officer_counts[1]) - np.sum(polya_urn_results['Counts']) into the second column
polya_urn_results['Counts'] = polya_urn_results['Counts'].apply(lambda x: np.concatenate(([np.sum(officer_counts[1]) - np.sum(x)], x)))
# add 1 to all the counts to match
polya_urn_results['Number of Complaints'] = polya_urn_results['Number of Complaints'].apply(lambda x: x + 1)

In [ ]:
# Remark, Data here is ascending, number of complaints seems like it gets capped, will need to look into this
polya_urn_results.head(50)

,Number of Complaints,Counts
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3318, 1123, 787, 613, 486, 439, 383, 297, 204..."
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3328, 1124, 768, 625, 520, 426, 347, 260, 223..."
2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3324, 1182, 737, 573, 540, 391, 345, 276, 239..."
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3308, 1163, 746, 611, 519, 405, 370, 282, 246..."
4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3246, 1218, 808, 611, 503, 421, 315, 272, 186..."
5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3292, 1229, 727, 577, 508, 421, 376, 271, 213..."
6,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3384, 1150, 710, 594, 507, 443, 345, 267, 211..."
7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3395, 1085, 711, 645, 512, 387, 386, 267, 215..."
8,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3340, 1133, 770, 601, 520, 408, 345, 262, 227..."
9,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3369, 1134, 759, 597, 461, 400, 373, 277, 250..."


In [ ]:
# explode the number of complaints column, then get max value
urn_max_complaints2 = max(polya_urn_results['Number of Complaints'].explode())
print(urn_max_complaints2)
# create dictionary of number of complaints and counts, key is number of complaints, value is counts
urn_complaints_dict2 = {}
for i in range(1, 11):
    urn_complaints_dict2[i] = []
# for each row in polya_urn_results, get the list of Number of Complaints, then get the list of Counts, then map them to the dictionary
for index, row in polya_urn_results.iterrows():
    # get list of number of complaints
    complaints = row['Number of Complaints']
    # get list of counts
    counts = row['Counts']
    # map them to the dictionary
    for i in range(len(complaints)):
        if complaints[i] < 10:
            urn_complaints_dict2[complaints[i]].append(counts[i])
        else:
            urn_complaints_dict2[10].append(counts[i])

# if the list length is not equal to num_simulations, then add 0s to the list until it is equal to num_simulations
for key in urn_complaints_dict2:
    if len(urn_complaints_dict2[key]) != num_simulations:
        urn_complaints_dict2[key] = urn_complaints_dict2[key] + [0] * (num_simulations - len(urn_complaints_dict2[key]))

45


In [ ]:
# get the average of each list in the dictionary
urn_avg_complaints_dict2 = {}
for key in urn_complaints_dict2:
    urn_avg_complaints_dict2[key] = np.mean(urn_complaints_dict2[key])

In [ ]:
print(urn_avg_complaints_dict2)

{1: 3329.566, 2: 1151.441, 3: 761.031, 4: 608.323, 5: 504.777, 6: 417.315, 7: 341.619, 8: 276.712, 9: 221.873, 10: 38.98964408350139}


In [ ]:
# get the 95% confidence interval of each list in the dictionary
upper_urn_ci_complaints_dict2 = {}
lower_urn_ci_complaints_dict2 = {}
for key in urn_complaints_dict2:
    # 95% confidence interval using quantile
    upper_urn_ci_complaints_dict2[key] = np.quantile(urn_complaints_dict2[key], 0.975)
    lower_urn_ci_complaints_dict2[key] = np.quantile(urn_complaints_dict2[key], 0.025)

In [ ]:
# plot curve
# Number of Complaints are on the x-axis, Counts are on the y-axis
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(urn_avg_complaints_dict2.keys()), y=list(urn_avg_complaints_dict2.values()), mode='lines', name='Spillover Polya Urn Model Results'))
# add line with color
fig.add_trace(go.Scatter(x=list(upper_urn_ci_complaints_dict2.keys()), y=list(upper_urn_ci_complaints_dict2.values()), mode='lines', name='Spillover Polya Urn Model 95% Confidence Interval', line=dict(color='rgb(66, 81, 245)', dash='dash')))
fig.add_trace(go.Scatter(x=list(lower_urn_ci_complaints_dict2.keys()), y=list(lower_urn_ci_complaints_dict2.values()), mode='lines', name='Spillover Polya Urn Model 95% Confidence Interval', line=dict(color='rgb(66, 81, 245)', dash='dash'), fill='tonexty', fillcolor='rgba(66, 81, 245,0.2)'))
fig.add_trace(go.Scatter(x=officer_counts[0], y=officer_counts[1], mode='lines', name='Data'))
fig.update_layout(title_text='Spillover Polya Urn Model Results')
fig.show()

In [ ]:
# plot curve
# Number of Complaints are on the x-axis, Counts are on the y-axis
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(perm_avg_complaints_dict.keys()), y=list(perm_avg_complaints_dict.values()), mode='lines', name='Permutation Test Results'))
# add line with color
fig.add_trace(go.Scatter(x=list(upper_perm_ci_complaints_dict.keys()), y=list(upper_perm_ci_complaints_dict.values()), mode='lines', name='Permutation Test 95% Confidence Interval', line=dict(color='rgb(235, 52, 58)', dash='dash')))
fig.add_trace(go.Scatter(x=list(lower_perm_ci_complaints_dict.keys()), y=list(lower_perm_ci_complaints_dict.values()), mode='lines', name='Permutation Test 95% Confidence Interval', line=dict(color='rgb(235, 52, 58)', dash='dash'), fill='tonexty', fillcolor='rgba(235, 52, 58,0.2)'))
fig.add_trace(go.Scatter(x=list(urn_avg_complaints_dict.keys()), y=list(urn_avg_complaints_dict.values()), mode='lines', name='Polya Urn Model Results'))
# add line with color
fig.add_trace(go.Scatter(x=list(upper_urn_ci_complaints_dict.keys()), y=list(upper_urn_ci_complaints_dict.values()), mode='lines', name='Polya Urn Model 95% Confidence Interval', line=dict(color='rgb(66, 81, 245)', dash='dash')))
fig.add_trace(go.Scatter(x=list(lower_urn_ci_complaints_dict.keys()), y=list(lower_urn_ci_complaints_dict.values()), mode='lines', name='Polya Urn Model 95% Confidence Interval', line=dict(color='rgb(66, 81, 245)', dash='dash'), fill='tonexty', fillcolor='rgba(66, 81, 245,0.2)'))
fig.add_trace(go.Scatter(x=officer_counts[0], y=officer_counts[1], mode='lines', name='Data'))
fig.add_trace(go.Scatter(x=list(urn_avg_complaints_dict2.keys()), y=list(urn_avg_complaints_dict2.values()), mode='lines', name='Spillover Polya Urn Model Results'))
# add line with color
fig.add_trace(go.Scatter(x=list(upper_urn_ci_complaints_dict2.keys()), y=list(upper_urn_ci_complaints_dict2.values()), mode='lines', name='Spillover Polya Urn Model 95% Confidence Interval', line=dict(color='rgb(100, 100, 200)', dash='dash')))
fig.add_trace(go.Scatter(x=list(lower_urn_ci_complaints_dict2.keys()), y=list(lower_urn_ci_complaints_dict2.values()), mode='lines', name='Spillover Polya Urn Model 95% Confidence Interval', line=dict(color='rgb(100, 100, 200)', dash='dash'), fill='tonexty', fillcolor='rgba(100, 100, 200,0.2)'))
fig.update_layout(title_text='Combined Results')
fig.show()